In [97]:
import pandas as pd

ratings = pd.read_csv('C:\IDE\Skillfactory\PY-12_Advanced_Pandas\practic\\ratings.csv', sep=',')
ratings

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy\t
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance\t
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller\t
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller\t
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller\t
...,...,...,...,...,...,...,...
100831,100831,610,166534,4.0,2017-05-03 21:53:22,Split (2017),Drama|Horror|Thriller\t
100832,100832,610,168248,5.0,2017-05-03 22:21:31,John Wick: Chapter Two (2017),Action|Crime|Thriller\t
100833,100833,610,168250,5.0,2017-05-08 19:50:47,Get Out (2017),Horror\t
100834,100834,610,168252,5.0,2017-05-03 21:19:12,Logan (2017),Action|Sci-Fi\t


In [98]:
# Для решения задач нам понадобится выделить из признака title год выпуска фильма. Для этого напишем функцию 
# get_year_release(arg).

#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [99]:
ratings['year_release'] = ratings['title'].agg(get_year_release)
ratings['year_release'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 100836 entries, 0 to 100835
Series name: year_release
Non-Null Count   Dtype  
--------------   -----  
100818 non-null  float64
dtypes: float64(1)
memory usage: 787.9 KB


In [100]:
mask = ratings['year_release'] == 1999
ratings[mask].groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [101]:
ratings[ratings['year_release'] == 2010].groupby('genres')['rating'].mean().sort_values().index[0]


'Action|Sci-Fi\t'

In [102]:
mask = ratings['year_release'] == 2010
ratings[mask].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi\t                        1.000000
Action|Adventure|Horror\t              1.500000
Action|Drama|Fantasy\t                 1.500000
Crime|Romance\t                        1.500000
Adventure|Comedy|Fantasy\t             1.833333
                                         ...   
Crime\t                                4.750000
Comedy|Musical\t                       5.000000
Animation|Drama|Fantasy|Mystery\t      5.000000
Adventure|Children|Comedy|Mystery\t    5.000000
Animation|Children|Mystery\t           5.000000
Name: rating, Length: 119, dtype: float64

In [103]:
x = ratings.groupby('userId')['genres'].nunique().sort_values(ascending=False)
x

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [104]:
ratings.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(by=['count', 'mean'], ascending=[True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [105]:
x = ratings[ratings['year_release'] == 2018].groupby('genres')['rating'].agg(['mean', 'count'])
x[x['count']>10].sort_values(by='mean', ascending=False)

,mean,count
genres,,
Action|Adventure|Sci-Fi\t,3.928571,14
Action|Comedy|Sci-Fi\t,3.875000,12


In [109]:

ratings['date'] = pd.to_datetime(ratings['date'])
ratings['year_rating'] = ratings['date'].dt.year
pivot = ratings.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
pivot

genres,(no genres listed)\t,Action\t,Action|Adventure\t,Action|Adventure|Animation\t,Action|Adventure|Animation|Children\t,Action|Adventure|Animation|Children|Comedy\t,Action|Adventure|Animation|Children|Comedy|Fantasy\t,Action|Adventure|Animation|Children|Comedy|IMAX\t,Action|Adventure|Animation|Children|Comedy|Romance\t,Action|Adventure|Animation|Children|Comedy|Sci-Fi\t,...,Romance|Thriller\t,Romance|War\t,Romance|Western\t,Sci-Fi\t,Sci-Fi|IMAX\t,Sci-Fi|Thriller\t,Sci-Fi|Thriller|IMAX\t,Thriller\t,War\t,Western\t
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000
1998,NaN,NaN,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.800000,NaN,NaN
1999,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000,NaN,NaN,NaN,NaN,4.000000,NaN,3.700000,4.5,4.000000
2000,NaN,2.588235,3.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.000,NaN,3.0,3.416667,NaN,2.142857,NaN,3.087912,3.0,4.058824
2001,NaN,3.000000,3.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,2.500000,NaN,2.500000,NaN,3.477273,3.0,3.111111
2002,NaN,2.750000,4.304348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000,NaN,NaN,3.750000,NaN,3.600000,NaN,3.583333,3.5,3.000000
2003,NaN,3.833333,3.277778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.375,2.5,NaN,2.333333,NaN,3.142857,NaN,3.250000,3.0,4.000000
2004,NaN,2.700000,4.136364,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,...,3.000,3.0,3.5,2.125000,NaN,NaN,NaN,3.464286,3.0,3.800000


In [ ]:
joined['date'] = pd.to_datetime(joined['date'])
joined['year_rating'] = joined['date'].dt.year
pivot = joined.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
display(pivot)

In [114]:
import pandas as pd
orders = pd.read_csv('C:\IDE\Skillfactory\PY-12 Advanced Pandas\practic\\orders.csv', sep='	')
products = pd.read_csv('C:\IDE\Skillfactory\PY-12 Advanced Pandas\practic\\products.csv', sep='	')
display(orders)
display(products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество
0,09.11.2019 21:55,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
1,09.11.2019 15:05,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100
2,09.11.2019 15:05,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10
3,09.11.2019 12:50,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7
4,09.11.2019 12:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5
5,09.11.2019 12:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5
6,08.11.2019 8:36,5,5,Отменён,Нет,Да,Нет,124,1
7,08.11.2019 8:36,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1
8,08.11.2019 8:36,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3
9,08.11.2019 8:36,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3


,Product_ID,Name,Price,CURRENCY
0,47,Шатны Полосатый рейс,2999,RUR
1,51,Платье Аленький цветочек,4999,RUR
2,53,Штаны Цветочная Поляна,4999,RUR
3,71,Платье Ночная Жизнь,7999,RUR
4,74,Платье Ночная Жизнь XXXL,8999,RUR
5,86,"Носки Простые, муж",45,RUR
6,91,"Носки Честные, муж",50,RUR
7,103,"Носки Подарочные, муж",199,RUR
8,104,"Носки Подарочные, жен",249,RUR
9,124,Носки беговые Camino,999,RUR


In [125]:
orders_products = orders.merge(
    products,
    right_on='Product_ID',
    left_on='ID товара',
    how='left'
)
orders_products

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 8:36,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 8:36,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 8:36,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 8:36,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


In [126]:
orders_products[orders_products['Name'].isna()]

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
17,01.01.2001 0:00,0,1,"Оплачен, формируется к отправке",Да,Нет,Нет,666,1,NaN,NaN,NaN,NaN


In [129]:
orders_products[orders_products['Статус'] == 'Отменён']

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
6,08.11.2019 8:36,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR


In [130]:
# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
orders_products['Profit'] = orders_products['Price'] * orders_products['Количество'] 
orders_products[orders_products['Оплачен'] == 'Да'].groupby('ID Покупателя')['Profit'].sum().sort_values(ascending=False)

ID Покупателя
7    17096.0
5    13043.0
8     1344.0
1        0.0
Name: Profit, dtype: float64